In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn import linear_model

In [2]:
def count_nan(input_df):
    x = (len(input_df) - input_df.count())/len(input_df)*100
    return x.sort_values()

In [3]:
#  Functions assisting in creation and visualisation of histogram 
# def hist_func(input_variable):
#     return int(input_variable/100)*100

# def clip_func(ip_variable, clip_value):
#     if ip_variable > clip_value:
#         ip_variable = clip_value
#     return ip_variable

In [4]:
prop = pd.read_csv("properties_2016.csv",low_memory=False)

In [5]:
#count_nan(prop)

In [6]:
## Return a dataframe with all the relevant columns
def process_dataframe(ip_dataframe):

    ## List of columns for mean_imputation
    mean_impute_columns = ['taxamount','calculatedfinishedsquarefeet',
                           'fullbathcnt','calculatedbathnbr','finishedsquarefeet12',
                          'buildingqualitytypeid','numberofstories','poolcnt','pooltypeid7',
                          'threequarterbathnbr','fireplacecnt','roomcnt','bedroomcnt','latitude',
                          'longitude','rawcensustractandblock']
    median_impute_columns = ['landtaxvaluedollarcnt', 'structuretaxvaluedollarcnt',
                             'taxvaluedollarcnt','lotsizesquarefeet','unitcnt','garagetotalsqft',
                            'garagecarcnt','regionidcounty','fips','bathroomcnt']
    mode_impute_columns = ['propertylandusetypeid','airconditioningtypeid',
                           'regionidzip','censustractandblock','yearbuilt','regionidcity',
                          'heatingorsystemtypeid','regionidneighborhood']
    
    ## Doing the imputation
    for col in median_impute_columns:
        median_for_this_col = ip_dataframe[col].median() 
        ip_dataframe[col].fillna(median_for_this_col, inplace = True)
    
    for col in mode_impute_columns:
        mode_for_this_col = ip_dataframe[col].mode()[0] ## Very important ## Took me 20 mins to debug
        ip_dataframe[col] = ip_dataframe[col].fillna(mode_for_this_col)
    
    for col in mean_impute_columns:
        mean_for_this_col = ip_dataframe[col].mean() 
        ip_dataframe[col].fillna(mean_for_this_col, inplace = True) 

    
    ## Columns to be used
    columns_to_be_used = median_impute_columns + mode_impute_columns + mean_impute_columns
    
    df_to_be_returned = ip_dataframe[columns_to_be_used]
    
    assert(count_nan(df_to_be_returned).sum() == 0)
    return df_to_be_returned

In [7]:
train_df = pd.read_csv('train_2016_v2.csv',parse_dates = ['transactiondate'])
#  Extracting the Transaction month out

train_df['transaction_month'] = train_df['transactiondate'].dt.month
print(train_df.describe())
print(train_df.head())

print(len(train_df))
print(train_df['parcelid'].nunique())

# Checking whether duplicate parcelids have different log error or not
# train_df[train_df['parcelid'].duplicated(keep=False)]


           parcelid      logerror  transaction_month
count  9.027500e+04  90275.000000       90275.000000
mean   1.298466e+07      0.011457           5.849848
std    2.504510e+06      0.161079           2.812690
min    1.071174e+07     -4.605000           1.000000
25%    1.155950e+07     -0.025300           4.000000
50%    1.254734e+07      0.006000           6.000000
75%    1.422755e+07      0.039200           8.000000
max    1.629608e+08      4.737000          12.000000
   parcelid  logerror transactiondate  transaction_month
0  11016594    0.0276      2016-01-01                  1
1  14366692   -0.1684      2016-01-01                  1
2  12098116   -0.0040      2016-01-01                  1
3  12643413    0.0218      2016-01-02                  1
4  14432541   -0.0050      2016-01-02                  1
90275
90150


In [8]:
# Left Join on train_df ## SLOW
raw_train_df = train_df.merge(prop, how='left', on = 'parcelid')

In [9]:
train_Y = raw_train_df['logerror'] 
raw_train_df.drop(labels= ['logerror', 'transactiondate', 'transaction_month'], axis = 1, inplace=True)

In [10]:
train_Y.describe()

count    90275.000000
mean         0.011457
std          0.161079
min         -4.605000
25%         -0.025300
50%          0.006000
75%          0.039200
max          4.737000
Name: logerror, dtype: float64

In [11]:
raw_train_df.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,1.0,NaN,NaN,2.0,3.0,NaN,4.0,2.0,NaN,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,1.0,NaN,NaN,3.0,2.0,NaN,4.0,3.0,NaN,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,1.0,NaN,NaN,2.0,2.0,NaN,4.0,2.0,NaN,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13


In [12]:
assert(len(raw_train_df) == len(train_df))

In [13]:
# Finding the different types of dtypes
type_df = raw_train_df.dtypes.reset_index()
type_df.columns = ['column_name','column_type']
type_df.groupby('column_type').aggregate('count').reset_index()


,column_type,column_name
0,int64,1
1,float64,52
2,object,5


In [14]:
test_data = pd.read_csv('sample_submission.csv')
test_data = test_data.rename(columns = {'ParcelId' : 'parcelid'})

# Left Join on test_data to import data from properties
raw_test_df = test_data.merge(prop,how='left',on = 'parcelid')
raw_test_df = raw_test_df.drop(['201610', '201611', '201612', '201710', '201711', '201712'],axis = 1)


In [15]:
assert((raw_train_df.columns == raw_test_df.columns).all())

In [16]:
final_train_df = process_dataframe(raw_train_df)
final_test_df = process_dataframe(raw_test_df)

from sklearn.linear_model import LinearRegression
# Creating linear regression object lm
lm = LinearRegression(normalize=True)

lm.fit(final_train_df, train_Y)

print (lm.score(final_train_df, train_Y))

print (lm.coef_)

print (final_train_df.columns)

my_prediction = lm.predict(final_train_df)

# We want mean absoulte error
print ('TRAIN MAE', np.mean(np.abs(my_prediction - train_Y)))
print ('TRAIN MSE', np.mean((my_prediction - train_Y)**2))

/usr/local/lib/python3.5/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


0.00523792533355
[ -4.78124731e-07  -4.69087887e-07   5.04688496e-07   6.24043751e-09
  -1.33441127e-03  -1.37554775e-05   2.02710926e-03   8.06349794e-06
   5.19903354e-03  -1.15501194e-02   2.71030603e-04   1.82023070e-04
  -3.46190534e-07   4.24677797e-14   3.81622915e-05  -3.61275044e-09
  -6.65470283e-04  -3.19731836e-09  -3.21552638e-06   4.49392083e-06
   2.91059703e-03   6.79393091e-03   6.91058881e-06   3.80601813e-04
  -2.58188886e-03  -1.84874250e-13  -5.07858208e-14   2.95449214e-02
   1.56803324e-03   3.29091004e-04   1.00320897e-03  -3.64313614e-09
   5.82310514e-09  -5.51336274e-07]
Index(['landtaxvaluedollarcnt', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'lotsizesquarefeet', 'unitcnt', 'garagetotalsqft',
       'garagecarcnt', 'regionidcounty', 'fips', 'bathroomcnt',
       'propertylandusetypeid', 'airconditioningtypeid', 'regionidzip',
       'censustractandblock', 'yearbuilt', 'regionidcity',
       'heatingorsystemtypeid', 'regionidneighborhood', 'ta

In [17]:
train_Y.describe()

count    90275.000000
mean         0.011457
std          0.161079
min         -4.605000
25%         -0.025300
50%          0.006000
75%          0.039200
max          4.737000
Name: logerror, dtype: float64

In [18]:
!%cat sample_submission.csv

/bin/sh: line 0: fg: no job control


In [19]:
np.abs(train_Y).mean()

0.06844671392965937

In [20]:
(train_Y**2).mean()

0.026077371669343673

In [21]:
raw_train_df.describe()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
count,9.027500e+04,90275.000000,261.000000,43.000000,90275.000000,90275.000000,16.0,90275.000000,90275.000000,658.0,...,95.000000,90275.000000,90275.000000,9.027500e+04,9.027500e+04,90275.0,9.027500e+04,90275.000000,1783.000000,9.027500e+04
mean,1.298466e+07,1.260271,7.229885,713.581395,2.279474,3.031869,4.0,5.565407,2.309216,66.0,...,311.694737,1968.419540,1.440739,1.798910e+05,4.576714e+05,2015.0,2.783344e+05,5983.975927,13.402692,6.049076e+13
std,2.504510e+06,1.721860,2.716196,437.434198,1.004271,1.156436,0.0,1.515046,0.969761,0.0,...,346.354850,23.695875,0.259909,2.087125e+05,5.548815e+05,0.0,4.004933e+05,6838.649682,2.715966,2.041793e+11
min,1.071174e+07,1.000000,2.000000,100.000000,0.000000,0.000000,4.0,1.000000,1.000000,66.0,...,18.000000,1885.000000,1.000000,1.000000e+02,2.200000e+01,2015.0,2.200000e+01,49.080000,6.000000,6.037101e+13
25%,1.155950e+07,1.000000,7.000000,407.500000,2.000000,2.000000,4.0,4.000000,2.000000,66.0,...,100.000000,1953.000000,1.440739,8.149000e+04,1.990235e+05,2015.0,8.222800e+04,2872.885000,13.000000,6.037400e+13
50%,1.254734e+07,1.000000,7.000000,616.000000,2.000000,3.000000,4.0,5.565407,2.000000,66.0,...,159.000000,1969.000000,1.440739,1.320000e+05,3.428720e+05,2015.0,1.929700e+05,4543.020000,14.000000,6.037620e+13
75%,1.422755e+07,1.000000,7.000000,872.000000,3.000000,4.000000,4.0,7.000000,3.000000,66.0,...,361.000000,1987.000000,1.440739,2.100425e+05,5.405890e+05,2015.0,3.454150e+05,6900.600000,15.000000,6.059042e+13
max,1.629608e+08,13.000000,21.000000,1555.000000,20.000000,16.000000,4.0,12.000000,20.000000,66.0,...,1366.000000,2015.000000,4.000000,9.948100e+06,2.775000e+07,2015.0,2.450000e+07,321936.090000,99.000000,6.111009e+13


In [22]:
for col in raw_train_df.columns:
    print(col, raw_train_df[col].nunique(dropna = False))

parcelid 90150
airconditioningtypeid 6
architecturalstyletypeid 7
basementsqft 40
bathroomcnt 23
bedroomcnt 17
buildingclasstypeid 2
buildingqualitytypeid 9
calculatedbathnbr 23
decktypeid 2
finishedfloor1squarefeet 1887
calculatedfinishedsquarefeet 5103
finishedsquarefeet12 4983
finishedsquarefeet13 12
finishedsquarefeet15 1916
finishedsquarefeet50 1899
finishedsquarefeet6 361
fips 3
fireplacecnt 6
fullbathcnt 15
garagecarcnt 15
garagetotalsqft 870
hashottuborspa 2
heatingorsystemtypeid 12
latitude 73312
longitude 71900
lotsizesquarefeet 20016
poolcnt 1
poolsizesum 274
pooltypeid10 2
pooltypeid2 2
pooltypeid7 1
propertycountylandusecode 78
propertylandusetypeid 14
propertyzoningdesc 1997
rawcensustractandblock 42647
regionidcity 177
regionidcounty 3
regionidneighborhood 494
regionidzip 388
roomcnt 17
storytypeid 2
threequarterbathnbr 5
typeconstructiontypeid 4
unitcnt 10
yardbuildingsqft17 568
yardbuildingsqft26 74
yearbuilt 130
numberofstories 5
fireplaceflag 2
structuretaxvaluedolla